# Elisa real estate, finding a model

Source: https://raw.githubusercontent.com/JulienAlardot/challenge-collecting-data/main/Data/database.csv

Libraries :
OneHotencoding
XGboost
Lasso
Ridge
Random forest
sklearn.preprocessing.LabelEncoder

Expected result: compare the prediction performance of the models, create best simulation and prediction

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
df = pd.read_csv('./data/may.csv', index_col=0)
df.head(4)

,Url,Source,Locality,Type of property,Subtype of property,Price,Type of sale,Number of rooms,Area,Fully equipped kitchen,...,Terrace Area,Garden,Garden Area,Surface of the land,Surface area of the plot of land,Number of facades,Swimming pool,State of the building,Province,Region
0,https://www.logic-immo.be/fr/vente/appartement...,logic-immo.be,2970,apartment,NaN,319799.0,regular sale,1.0,NaN,0.0,...,11.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,Antwerp,Vlaams
1,https://www.logic-immo.be/fr/vente/appartement...,logic-immo.be,2970,apartment,NaN,291999.0,regular sale,1.0,NaN,0.0,...,6.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,Antwerp,Vlaams
2,https://www.logic-immo.be/fr/vente/appartement...,logic-immo.be,2970,apartment,NaN,764999.0,regular sale,2.0,153.0,0.0,...,62.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,Antwerp,Vlaams
3,https://www.logic-immo.be/fr/vente/appartement...,logic-immo.be,2970,apartment,NaN,660264.0,regular sale,3.0,NaN,0.0,...,160.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,Antwerp,Vlaams


## Data cleaning

 - Existence of duplicated columns ?
 - Question: Get rid of title column ?
 - Remove URL column which is there for debugging purpose
 - Remove columns with missing values with threshold more than 80% Nan or equivalent
 - Check if the type of data is homogenous per column if not clean
 - Check abnomalies and outliers
 

In [2]:
# check duplicates
df.duplicated().any()

False

In [4]:
# get rid of columns with less than 20% data
df.dropna(thresh=len(df)*0.8, axis='columns', inplace = True)
df.head(4)

,Source,Locality,Type of property,Price,Type of sale,Number of rooms,Area,Fully equipped kitchen,Open fire,Terrace,Garden,Swimming pool,Province,Region
0,logic-immo.be,2970,apartment,319799.0,regular sale,1.0,NaN,0.0,0.0,1.0,0.0,0.0,Antwerp,Vlaams
1,logic-immo.be,2970,apartment,291999.0,regular sale,1.0,NaN,0.0,0.0,1.0,0.0,0.0,Antwerp,Vlaams
2,logic-immo.be,2970,apartment,764999.0,regular sale,2.0,153.0,0.0,0.0,1.0,0.0,0.0,Antwerp,Vlaams
3,logic-immo.be,2970,apartment,660264.0,regular sale,3.0,NaN,0.0,0.0,1.0,0.0,0.0,Antwerp,Vlaams


In [14]:
# insert filters from colab
df = df[(df.Price >= 50000) & (df.Price <= 10000000)]
df.sort_values(by='Price', ascending = False).head(15)
# todo equivalent of inplace when working on columns

,Source,Locality,Type of property,Price,Type of sale,Number of rooms,Area,Fully equipped kitchen,Open fire,Terrace,Garden,Swimming pool,Province,Region
62679,immoweb,7760,house,10000000.0,regular sale,90.0,3600.0,0.0,0.0,0.0,0.0,0.0,Hainaut,Wallonie
29978,immoweb,1050,house,10000000.0,regular sale,0.0,NaN,0.0,0.0,0.0,0.0,0.0,Brussels,Brussels Capital
52601,immoweb,9840,house,9876543.0,regular sale,9.0,2500.0,0.0,0.0,0.0,0.0,0.0,Oost-Vlanderen,Vlaams
35094,immoweb,1950,house,9500000.0,regular sale,12.0,1100.0,0.0,0.0,1.0,0.0,1.0,Vlaams-Brabant,Vlaams
56420,immoweb,8300,house,8500000.0,regular sale,4.0,591.0,0.0,0.0,1.0,1.0,1.0,West-Vlanderen,Vlaams
19286,immoweb,4000,house,7490000.0,regular sale,47.0,NaN,0.0,0.0,0.0,0.0,0.0,Liège,Wallonie
31756,immoweb,1050,house,7300000.0,regular sale,0.0,1950.0,0.0,1.0,0.0,0.0,0.0,Brussels,Brussels Capital
26988,immoweb,1050,house,7000000.0,regular sale,40.0,NaN,0.0,0.0,0.0,0.0,0.0,Brussels,Brussels Capital
41617,immoweb,1050,house,6700000.0,regular sale,46.0,1600.0,0.0,0.0,0.0,0.0,0.0,Brussels,Brussels Capital
47012,immoweb,1380,house,6500000.0,regular sale,9.0,1746.0,0.0,0.0,1.0,0.0,0.0,Brabant Wallon,Wallonie


In [13]:
# visually 


In [3]:
# get rid of Url before training
# df.drop(['Url'], axis=1, inplace = True)
# df.head(4)

,Source,Locality,Type of property,Subtype of property,Price,Type of sale,Number of rooms,Area,Fully equipped kitchen,Furnished,...,Terrace Area,Garden,Garden Area,Surface of the land,Surface area of the plot of land,Number of facades,Swimming pool,State of the building,Province,Region
0,logic-immo.be,2970,apartment,NaN,319799.0,regular sale,1.0,NaN,0.0,0.0,...,11.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,Antwerp,Vlaams
1,logic-immo.be,2970,apartment,NaN,291999.0,regular sale,1.0,NaN,0.0,0.0,...,6.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,Antwerp,Vlaams
2,logic-immo.be,2970,apartment,NaN,764999.0,regular sale,2.0,153.0,0.0,0.0,...,62.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,Antwerp,Vlaams
3,logic-immo.be,2970,apartment,NaN,660264.0,regular sale,3.0,NaN,0.0,0.0,...,160.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,Antwerp,Vlaams


TODO
- polynmial
- random forest
- xgboost
